In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
from statistics import *
import numpy as np

Using TensorFlow backend.


In [2]:
# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # reshape dataset to have a single channel
    #the pixel values are grayscale, the pixel dimension is set to 1.
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

In [3]:
trainX, trainY, testX, testY = load_dataset()
input_shape = (28, 28, 1)

In [4]:
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    # Making sure that the values are float so that we can get decimal points after division
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    # Normalizing the grey scale codes by dividing it by the max grey scale value.
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

In [5]:
trainX, testX = prep_pixels(trainX, testX)

In [6]:
def generate_noise_data(noise_factor):
    x_train_noisy = trainX+ noise_factor * np.random.normal(loc=0.0, scale=1.0, size=trainX.shape)
    x_test_noisy = testX + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=testX.shape)
    x_train_noisy = np.clip(x_train_noisy, 0., 1.)
    x_test_noisy = np.clip(x_test_noisy, 0., 1.)
    return x_train_noisy, x_test_noisy

In [7]:
def model_build_fit_evaluate(x_train_noisy, x_test_noisy, trainY, testY):
    # deep learning models are created where an instance of the Sequential class is created
    # model layers are created and added to it.
    model = Sequential() 

    # 28 filters, each being size of 3x3 and slides through the image  with a stride of magnitude 1 in horizontal and vertical direction using ReLU activation function
    model.add(Conv2D(28, kernel_size=(3,3), strides=(1,1), input_shape=input_shape)) 

    #adding batch normalization to changing the distribution by standarising the output
    model.add(BatchNormalization())

    #max pooling to extract digit features
    model.add(MaxPooling2D(pool_size=(2,2)))

    #increasing the model depth
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', strides=(1,1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', strides=(1,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #turns matrix values into a single vector
    model.add(Flatten()) 
    #hidden layer with 200 nodes and relu activation function
    model.add(Dense(200,activation = "relu")) 
    # dropout regularization to drop certain neurons to reduce overfitting
    model.add(Dropout(0.3)) 
    #adding batch normalization to changing the distribution by standarising the output
    model.add(BatchNormalization())
    #output layer for classification values of digits 0-9
    model.add(Dense(10,activation = "softmax"))
    # compile model
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train_noisy, trainY, epochs=10, validation_data=(x_test_noisy, testY))
    model.evaluate(x_test_noisy, testY, verbose=0)

In [8]:
noise_factor = 0.1
x_train_noisy_new, x_test_noisy_new = generate_noise_data(noise_factor)
model_build_fit_evaluate(x_train_noisy_new, x_test_noisy_new, trainY, testY)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 110s 2ms/step - loss: 0.1538 - accuracy: 0.9554 - val_loss: 0.0581 - val_accuracy: 0.9808
Epoch 2/10
60000/60000 [==============================] - 111s 2ms/step - loss: 0.0635 - accuracy: 0.9808 - val_loss: 0.0671 - val_accuracy: 0.9779
Epoch 3/10
60000/60000 [==============================] - 112s 2ms/step - loss: 0.0456 - accuracy: 0.9859 - val_loss: 0.0281 - val_accuracy: 0.9915
Epoch 4/10
60000/60000 [==============================] - 112s 2ms/step - loss: 0.0389 - accuracy: 0.9884 - val_loss: 0.0352 - val_accuracy: 0.9895
Epoch 5/10
60000/60000 [==============================] - 112s 2ms/step - loss: 0.0325 - accuracy: 0.9896 - val_loss: 0.0275 - val_accuracy: 0.9914
Epoch 6/10
60000/60000 [==============================] - 112s 2ms/step - loss: 0.0257 - accuracy: 0.9918 - val_loss: 0.0314 - val_accuracy: 0.9909
Epoch 7/10
60000/60000 [==============================] - 110s

In [9]:
noise_factor = 0.25
x_train_noisy_new, x_test_noisy_new = generate_noise_data(noise_factor)
model_build_fit_evaluate(x_train_noisy_new, x_test_noisy_new, trainY, testY)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 120s 2ms/step - loss: 0.1857 - accuracy: 0.9460 - val_loss: 0.0558 - val_accuracy: 0.9817
Epoch 2/10
60000/60000 [==============================] - 117s 2ms/step - loss: 0.0745 - accuracy: 0.9771 - val_loss: 0.0449 - val_accuracy: 0.9848
Epoch 3/10
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0568 - accuracy: 0.9827 - val_loss: 0.0406 - val_accuracy: 0.9875
Epoch 4/10
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0434 - accuracy: 0.9857 - val_loss: 0.0351 - val_accuracy: 0.9895
Epoch 5/10
60000/60000 [==============================] - 115s 2ms/step - loss: 0.0374 - accuracy: 0.9880 - val_loss: 0.0447 - val_accuracy: 0.9858
Epoch 6/10
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0319 - accuracy: 0.9897 - val_loss: 0.0380 - val_accuracy: 0.9885
Epoch 7/10
60000/60000 [==============================] - 117s

In [10]:
noise_factor = 0.50
x_train_noisy_new, x_test_noisy_new = generate_noise_data(noise_factor)
model_build_fit_evaluate(x_train_noisy_new, x_test_noisy_new, trainY, testY)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 109s 2ms/step - loss: 0.3037 - accuracy: 0.9057 - val_loss: 0.1210 - val_accuracy: 0.9611
Epoch 2/10
60000/60000 [==============================] - 110s 2ms/step - loss: 0.1496 - accuracy: 0.9538 - val_loss: 0.1044 - val_accuracy: 0.9660
Epoch 3/10
60000/60000 [==============================] - 99s 2ms/step - loss: 0.1227 - accuracy: 0.9609 - val_loss: 0.0909 - val_accuracy: 0.9703
Epoch 4/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.1039 - accuracy: 0.9669 - val_loss: 0.0864 - val_accuracy: 0.9709
Epoch 5/10
60000/60000 [==============================] - 123s 2ms/step - loss: 0.0923 - accuracy: 0.9700 - val_loss: 0.0834 - val_accuracy: 0.9723
Epoch 6/10
60000/60000 [==============================] - 165s 3ms/step - loss: 0.0826 - accuracy: 0.9736 - val_loss: 0.0771 - val_accuracy: 0.9750
Epoch 7/10
60000/60000 [==============================] - 141s 

In [11]:
noise_factor = 0.75
x_train_noisy_new, x_test_noisy_new = generate_noise_data(noise_factor)
model_build_fit_evaluate(x_train_noisy_new, x_test_noisy_new, trainY, testY)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.5471 - accuracy: 0.8231 - val_loss: 0.3210 - val_accuracy: 0.8947
Epoch 2/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.3645 - accuracy: 0.8827 - val_loss: 0.3097 - val_accuracy: 0.9035
Epoch 3/10
60000/60000 [==============================] - 109s 2ms/step - loss: 0.3138 - accuracy: 0.8987 - val_loss: 0.3005 - val_accuracy: 0.9006
Epoch 4/10
60000/60000 [==============================] - 109s 2ms/step - loss: 0.2960 - accuracy: 0.9029 - val_loss: 0.2505 - val_accuracy: 0.9166
Epoch 5/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.2649 - accuracy: 0.9128 - val_loss: 0.2494 - val_accuracy: 0.9142
Epoch 6/10
60000/60000 [==============================] - 111s 2ms/step - loss: 0.2544 - accuracy: 0.9169 - val_loss: 0.2587 - val_accuracy: 0.9145
Epoch 7/10
60000/60000 [==============================] - 116s